In [ ]:
import os, sys, numpy as np, torch, matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
ROOT_DIR = os.path.abspath("..")
DATA_DIR = os.path.join(ROOT_DIR, "data", "processed")
CKPT_DIR = os.path.join(ROOT_DIR, "results", "checkpoints")
VIS_DIR  = os.path.join(ROOT_DIR, "results", "visualizations")
os.makedirs(VIS_DIR, exist_ok=True)

In [ ]:
sys.path.append(ROOT_DIR)
from core.model import TumorNet34Bayes
from core.utils import compute_metrics, ece_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = TumorNet34Bayes(dropout_p=0.4).to(device)
model.load_state_dict(torch.load(
    os.path.join(CKPT_DIR, "model_weights_best.pth"),  
    map_location=device
))
model.eval()

In [ ]:
X_test = np.load(os.path.join(DATA_DIR, "X_test.npy"))
y_test = np.load(os.path.join(DATA_DIR, "y_test.npy"))

X = torch.tensor(X_test, dtype=torch.float32).permute(0,3,1,2).to(device)

In [ ]:
B = X.size(0)
probs_list = []  

with torch.no_grad():
    for i in range(0, B, 32):
        xb = X[i:i+32]
        logits, model_probs, conf = model(xb) 
        p = torch.sigmoid(logits).cpu().numpy().ravel().tolist()
        probs_list.extend(p)

In [ ]:
labels = y_test.astype(int)
metrics = compute_metrics(labels, np.array(probs_list), threshold=0.5)
print("Metrics:", metrics)
print("ECE:", ece_score(np.array(probs_list), labels, n_bins=15))

In [ ]:
fpr, tpr, _ = roc_curve(labels, probs_list)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, lw=2, label=f"AUC={roc_auc:.3f}")
plt.plot([0,1],[0,1],'--', lw=1)
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.savefig(os.path.join(VIS_DIR, "roc_curve.png"))
plt.show()